In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Spatial Inequality Dynamics



In [ ]:
import seaborn as sns 
import pandas as pd
import geopandas as gpd
import pysal 
import numpy as np
import mapclassify
import matplotlib.pyplot as plt
from pysal.explore import esda
from pysal.lib import weights

## Data: U.S. state per capita income 1969-2017

For this chapter, we use data on average income per capita over time. Specifically, we consider the United States counties from 1969 to 2017. The U.S. counties are small regions that fit hierarchically within states. This perspective will allow us to examine trends for individual observations (counties), or regions containing several of them in a geographically consistent way (states or census regions which are collections of states). The temporal approach will reveal whether these entities get richer or poorer, as well as how the overall distribution of income moves, skews, or spreads out. 


In [ ]:
pci_df = gpd.read_file('https://www.dropbox.com/s/4ihybylqx9b8hun/uscountypcincome.gpkg?dl=1')
pci_df.columns

In [ ]:
pci_df.head()


In this dataset, we have 3076 counties across 49 years, as well as 28 extra columns that describe each county. 



In [ ]:
pci_df.shape

As an example, we can see the first ten years for Jackson County, Mississippi (state code `28`) below:

In [ ]:
pci_df.query('NAME == "Jackson" & STATEFP == "28"').loc[
    :, "1969":"1979"
]

## Global inequality

We begin our examination of inequality by focusing on several global measures of income inequality. Here, "global" means that the measure is concerned with the overall nature of inequality within the income distribution. That is, these measures focus on the direct disparity between rich and poor, considering nothing about where the rich and poor live. Several classic measures of inequality are available for this purpose. 

In general terms, measures of inequality focus on the dispersion present in an income distribution. In the case of regional or spatial inequality, the distributions describe the average or per capita incomes for spatial units, such as for counties, census tracts, or regions. For our U.S. county data, we can visualize (Fig. XXX1XXX) the distribution of per capita incomes for the first year in the sample as follows:

In [ ]:
sns.histplot(x=pci_df["1969"], kde=True);

Looking at this distribution, notice that the right side of the distribution is much longer than the left side. This long right tail is a prominent feature, and is common in the study of incomes and many other societal phenomena, as it reflects the fact that within a single income distribution, the super-rich are generally much more wealthy than the super-poor are deprived, compared to the average. 


Before we can get to mapping, we change the CRS to a suitable one for mapping, the Albers Equal Area projection for North America:  

In [ ]:
pci_df = pci_df.to_crs(
    # Albers Equal Area North America
    epsg=5070
)

And the quantile choropleth for 1969 (Fig. XXX2XXX) can be generated by:

In [ ]:
ax = pci_df.plot(
    column="1969",
    scheme="Quantiles",
    legend=True,
    edgecolor="none",
    legend_kwds={"loc": "lower left"},
    figsize=(12, 12),
)
ax.set_axis_off()
plt.show()


### 20:20 ratio

In [ ]:
top20, bottom20 = pci_df["1969"].quantile([0.8, 0.2])

In [ ]:
top20

In [ ]:
bottom20

In [ ]:
top20 / bottom20

In [ ]:
top20, bottom20 = pci_df["2017"].quantile([0.8, 0.2])

In [ ]:
top20 / bottom20

Calculate how the 20:20 Ratio changes over time

In [ ]:
def ineq_20_20(values):
    top20, bottom20 = values.quantile([0.8, 0.2])
    return top20 / bottom20


# Generate range of strings from 1969 to 2018
years = np.arange(1969, 2018).astype(str)

# Compute 20:20 ratio for every year
ratio_2020 = pci_df[years].apply(ineq_20_20, axis=0)

# Plot evolution of 20:20 ratio
ax = plt.plot(years, ratio_2020)

# Grab figure generated in the plot
figure = plt.gcf()
# Replace tick labels with every other year
plt.xticks(years[::2])
# Set vertical label
plt.ylabel("20:20 ratio")
# Set horizontal label
plt.xlabel("Year")
# Rotate year labels
figure.autofmt_xdate(rotation=45)

plt.show()

In addition to the 20:20 ratio, we will explore two more traditional measures of inequality: the Gini index and Theil's index. For these, we will use the `inequality` package from Pysal.

In [ ]:
from pysal.explore import inequality

### Gini index and Lorenz Curve
We can construct the Lorenz curve for 1969 by first computing the share of our population of counties that is below each observation. For that, we generate a cumulative series:

In [ ]:
n = len(pci_df)
share_of_population = np.arange(1, n + 1) / n
## This is an equally spaced array of numbers

Then, we consider the cumulative evolution of income. For this, we need to find out the proportion of total income owned by each share of the population. Empirically, this can be computed in the following fashion. First, we sort county incomes:

In [ ]:
incomes = pci_df["1969"].sort_values()

Second, we find the overall percentage of income accumulated at each data point. To do this, we compute what percentage of the total income each county represents:

In [ ]:
shares = incomes / incomes.sum()

In [ ]:
shares

and construct the *cumulative sum* of these shares, which reflects the sum of all of the shares of income up to the current one:

$$ \texttt{CumSum(v, k)} = \sum_{i=1}^k v_i$$

This starts at $0$ and reaches $1$ once the last share is included:

In [ ]:
cumulative_share = shares.cumsum()

With this, we can plot both the Lorenz curve and the line of perfect equality (Fig. XXX4XXX):

In [ ]:
# Generate figure with one axis
f, ax = plt.subplots()
# Plot Lorenz Curve
ax.plot(share_of_population, cumulative_share, label="Lorenz Curve")
# Plot line of perfect equality
ax.plot((0, 1), (0, 1), color="r", label="Perfect Equality")
# Label horizontal axis
ax.set_xlabel("Share of population")
# Label vertical axis
ax.set_ylabel("Share of income")
# Add legend
ax.legend()

plt.show()

The blue line is the Lorenz curve for county incomes in 1969. The Gini index is the area between it and the 45-degree line of equality shown in red, all standardized by the area underneath the line of equality.

A first approach to examine how inequality has evolved is to plot the Lorenz curves for each year. One way to do this in Python involves creating a function that will compute the Lorenz curve for an arbitrary set of incomes. The following function encapsulates the steps shown above into a single shot:

In [ ]:
def lorenz(y):
    y = np.asarray(y)
    incomes = np.sort(y)
    income_shares = (incomes / incomes.sum()).cumsum()
    N = y.shape[0]
    pop_shares = np.arange(1, N + 1) / N
    return pop_shares, income_shares

For a single year, say 1969, our function would return a tuple with two arrays, one for each axis in the Lorenz curve plot:

In [ ]:
lorenz(pci_df["1969"])

## Gini inequality in Pysal

In [ ]:
g69 = inequality.gini.Gini(pci_df["1969"].values)

To extract the coefficient, we retrieve the `g` property of `g69`:

In [ ]:
g69.g

Here, the Gini coefficient in 1969 was 0.13. To compute this for every year, we can use a similar pattern as we have before. First, define a function to compute the quantity of interest; then, apply the function across the table with all years:

In [ ]:
def gini_by_col(column):
    return inequality.gini.Gini(column.values).g

`inequality`'s Gini requires an `numpy.ndarray` rather than a `pandas.Series` object, which we can pull out through the `values` attribute. This is passed to the `Gini` class, and we only return the value of the coefficient as a `DataFrame` object.

In [ ]:
inequalities = (
    pci_df[years].apply(gini_by_col, axis=0).to_frame("gini")
)

In [ ]:
inequalities

This results in a series of Gini values, one for each year:

In [ ]:
inequalities.head()

Which we can turn into a graphical representation through standard `pandas` plotting. The resulting pattern (Fig. XXX6XXX) is similar to that of the 20:20 ratio above:

In [ ]:
inequalities.plot(figsize=(10, 3));

### Theil's index

A third commonly used measure of inequality is Theil's $T$ {cite}`manz2021` given as:

$$T = \sum_{i=1}^m \left( \frac{y_i}{\sum_{i=1}^m y_i} \ln \left[ m \frac{y_i}{\sum_{i=1}^m y_i}\right] \right)$$

where $y_i$ is per capita income in area $i$ among $m$ areas. Conceptually, this metric is related to the entropy of the income distribution, measuring how evenly distributed incomes are across the population.

The Theil index is also available in Pysal's `inequality`, so we can take a similar approach as above to calculate it for every year:

In [ ]:
def theil(column):
    return inequality.theil.Theil(column.values).T


inequalities["theil"] = pci_df[years].apply(theil, axis=0)

And generate a plot of its evolution over time (Fig. XXX7XXX):

In [ ]:
inequalities["theil"].plot(color="orange", figsize=(10, 3));

The time paths of the Gini and the Theil coefficients appear to show striking
similarities. At first glance, this might suggest that the indices are
substitutes for one another. However, if we plot them against each other (Fig. XXX8XXX), we can see they are not perfectly correlated: 

In [ ]:
_ = sns.regplot(x="theil", y="gini", data=inequalities)

Indeed, as we shall see below, each index has
properties that lend themselves to particular spatial extensions that work in complementary ways. We need both (and more) for a complete picture. 



## Spatial inequality

The analysis of regional income inequality differs from the analysis of
national interpersonal income inequality in its focus on spatial units. Since
regional incomes are explicitly embedded in geographical space, we can take advantage
of their spatial configuration to learn more about the nature of the inequality.
In the regional inequality literature, this has been approached in a number of ways.


### Spatial autocorrelation



This approach helps us shed light on the properties of the spatial pattern of regional income data. We return to global measures of spatial autocorrelation that we encountered earlier in the book. The essence of this approach is to examine to what extent the spatial distribution of incomes is concentrated over space. For this, we use a queen spatial weights matrix and calculate Moran's I for
each year in the sample:

In [ ]:
wq = weights.Queen.from_dataframe(pci_df)

Following the same pattern to "broadcast" a function, we create a function that returns the results we need from each statistic. Here, we will also keep the pseudo $p$-value for the Moran statistic which, as we saw in [Chapter 6](06_spatial_autocorrelation), helps us identify whether the index is statistically significant under the null hypothesis that incomes are randomly distributed geographically.

In [ ]:
def moran_by_col(y, w=wq):
    mo = esda.Moran(y, w=w)
    mo_s = pd.Series(
        {"I": mo.I, "I-P value": mo.p_sim},
    )
    return mo_s

This time, our function returns a `Series` object so that when we pass it through `apply`, we get a well-formatted table:

In [ ]:
moran_stats = pci_df[years].apply(moran_by_col, axis=0).T

moran_stats.head()

For further comparison, the results are attached to the `inequalities` table:

In [ ]:
inequalities = inequalities.join(moran_stats)

Which can be visualized (Fig. XXX9XXX) by:

In [ ]:
inequalities[["I", "I-P value"]].plot(subplots=True, figsize=(10, 6))
plt.show()

This suggests a gradual decline in the geographic structure of inequality with two implications: 
- (a) per capita incomes are now less similar between nearby counties and 
- (b), this has been consistently declining, regardless of whether inequality is high or low. 
Second, despite this decline, there is never a year in which the spatial autocorrelation is not statistically significant. In other words, there is a strong geographic structure in the distribution of regional incomes that needs to be accounted for when focusing on inequality questions.


## Q.1
Calculate the change in spatial autocorrelation using a different weight, such as: 
- Kernel density
- KNN

## Segregation
We are now going to look at segregation. Let's read a new dataset from the 2018 ACS for NYC

In [ ]:
acs_2018 = gpd.read_file('https://www.dropbox.com/s/433kt7spgslphvu/acs_race_income_2018_new.zip?dl=1')
acs_2018 = acs_2018.rename(columns={ 'total_pop':'total_pop', 
                          'total_pop_':'total_pop_hispanic', 
                          'total_po_1':'total_pop_white',
                          'total_po_2':'total_pop_black', 
                         'total_po_3':'total_pop_native', 
                         'total_po_4':'total_pop_asian',
                           'total_po_5':'total_pop_pacific', 
                         'total_po_6':'total_pop_other', 
                         'median_hh_':'median_hh_income'})

In [ ]:
acs_2018

In [ ]:
acs_2018.rename(columns={ 'total_pop':'total_pop', 
                          'total_pop_':'total_pop_hispanic', 
                          'total_po_1':'total_pop_white',
                          'total_po_2':'total_pop_black', 
                         'total_po_3':'total_pop_native', 
                         'total_po_4':'total_pop_asian',
                           'total_po_5':'total_pop_pacific', 
                         'total_po_6':'total_pop_other', 
                         'median_hh_':'median_hh_income'})

In [ ]:
acs_2018.head()

In [ ]:
acs_2018.plot('total_pop_white')

In [ ]:
acs_2018.info()

In [ ]:
from pysal.explore import segregation

### Aspatial Dissimilarity
The standard (aspatial) formula for the Dissimilarity index is: 
$$
D = \frac{1}{2}\sum_i^N {\lvert \frac{a_i}{A} - \frac{b_i}{B} \lvert}
$$

where $a_i$ is the population of group $a$ in tract $i$, $A$ is the total population for group $a$ in the city. $b_i$ is the population for group $b$ in tract $i$ and $B$ is the total population for group $b$.

This is how the Dissimilarity index is implemented in pysal. It assumes a main group of interest and the group "B" is the rest of the population.

In [ ]:
dissim = segregation.singlegroup.Dissim(acs_2018, group_pop_var="total_pop_white",total_pop_var="total_pop")

In [ ]:
dissim.statistic


### Entropy / Theil index
 $$T=∑_i^m (\frac{y_i}{\sum_i^m y_i}ln[m\frac{y_i}{\sum_i^m y_i}])$$

Where $y_i$ is the number of minority group residents in each subarea.



In [ ]:
entropy = segregation.singlegroup.Entropy(acs_2018, group_pop_var="total_pop_black",total_pop_var="total_pop")
entropy.statistic

### Isolation
 $$I=∑_i \frac{x_i}{X}\frac{x_i}{t_i}$$

Where $x_i$ is the number of minority group residents (Black, Hispanic, Asian) in each subarea, $X$ is the number of minority group residents in the region, and $t_i$ is the total population of the region.


In [ ]:
isolation = segregation.singlegroup.Isolation(acs_2018, group_pop_var="total_pop_black",total_pop_var="total_pop")
isolation.statistic

### Multigroup Dissimilarity
And there are multi-group variants of these measures. Here is the dissimilarity index.

In [ ]:
multi_dissim = segregation.multigroup.MultiDissim(acs_2018, 
                                                  groups=['total_pop_white', 'total_pop_black', 'total_pop_hispanic','total_pop_asian'])


In [ ]:
multi_dissim.statistic

## Q.2 
Calculate the black-white dissimilarity for `acs_2018`

### Spatial measures (explicit)

In [ ]:
cent = segregation.singlegroup.AbsoluteCentralization(acs_2018, group_pop_var="total_pop_black",total_pop_var="total_pop")

In [ ]:
cent.statistic


### Spatial measures (implicit)
#### Euclidian distance based measures
We are using a kernel function with `distance` as the kernel bandwidth.

In [ ]:
aspatial_gini = segregation.singlegroup.Gini(acs_2018, group_pop_var="total_pop_white",total_pop_var="total_pop")

In [ ]:
aspatial_gini.statistic

In [ ]:
## Make sure your data is in the right projection so that the distance is in meters
gen_spatialgini =  segregation.singlegroup.Gini(acs_2018, group_pop_var="total_pop_white",total_pop_var="total_pop", distance=2000)


In [ ]:
gen_spatialgini.statistic


## Local Measures of Segregation


In [ ]:
acs_2018.columns

In [ ]:
groups_list = ['total_pop_hispanic', 'total_pop_white',
       'total_pop_black', 'total_pop_native', 'total_pop_asian',
       'total_pop_pacific', 'total_pop_other',]

### Local entropy

In [ ]:
entropy_local = segregation.local.MultiLocalEntropy(acs_2018, groups_list,)

In [ ]:
entropy_local.statistics

In [ ]:
acs_2018['local_entropy'] = entropy_local.statistics

In [ ]:
acs_2018.plot('local_entropy', legend=True,cmap='inferno_r')

### Local Centralization
(This might take a few minutes!)

In [ ]:
centralization_local = segregation.local.LocalRelativeCentralization(acs_2018, 'total_pop_black', 'total_pop')
acs_2018['local_centralization'] = centralization_local.statistics

In [ ]:
acs_2018.plot('local_centralization', legend=True,cmap='inferno_r')

### Local Simpson Interaction
The is a measure of the probability that two people in a unit (tract here), have the probability of interacting

In [ ]:
simpson = segregation.local.MultiLocalSimpsonInteraction(acs_2018, groups_list)
acs_2018['local_simpsoninteraction'] = simpson.statistics

In [ ]:
acs_2018.plot('local_simpsoninteraction', legend=True,cmap='inferno_r')

## Spatial lags 
Almost all these indices have an option to include a spatial lag. 

In [ ]:
wknn = weights.KNN.from_dataframe(acs_2018,k=5)

In [ ]:
entropy_spatial = segregation.local.MultiLocalEntropy(acs_2018, groups_list,w=wknn)
acs_2018['local_entropy_spatial'] = entropy_spatial.statistics

In [ ]:
acs_2018.plot('local_entropy_spatial', legend=True,cmap='inferno_r')

In [ ]:
simpson_spatial = segregation.local.MultiLocalSimpsonInteraction(acs_2018, groups_list,w=wknn)
acs_2018['local_simpsoninteraction_spatial'] = simpson_spatial.statistics

In [ ]:
acs_2018.plot('local_simpsoninteraction_spatial', legend=True,cmap='inferno_r')